<a href="https://colab.research.google.com/github/y-kamiya/machine-learning-samples/blob/feature%2Ftts-scripts/scripts/tts/TTS_jsut_glowtts_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Hands-on example for 🐸 [Coqui TTS](https://github.com/coqui-ai/TTS)

original notebook: https://colab.research.google.com/gist/erogol/97516ad65b44dbddb8cd694953187c5b/tts_example.ipynb

In [ ]:
!nvidia-smi

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
!ls /gdrive

!ln -s "/gdrive/My Drive" /mydrive
DATAROOT='/mydrive/machine-learning/tts/data'

In [ ]:
!sudo apt-get install automake libtool
!git clone https://github.com/tset-tset-tset/espeak-ng -b tset-tset-tset-patch-1
%cd espeak-ng/
!./autogen.sh && ./configure --libdir=/usr/lib/x86_64-linux-gnu
!make
!sudo make install

## 以下がゴミなく変換できれば成功
!echo 'スウェーデン' | espeak-ng -x -v ja

%cd ..

In [ ]:
# get TTS to your local
!git clone https://github.com/coqui-ai/TTS
%cd TTS

!git checkout 4132240
!pip install -r requirements.txt
!python setup.py develop

Cloning into 'TTS'...
remote: Enumerating objects: 15070, done.
remote: Counting objects: 100% (1485/1485), done.
remote: Compressing objects: 100% (506/506), done.
remote: Total 15070 (delta 1005), reused 1356 (delta 963), pack-reused 13585
Receiving objects: 100% (15070/15070), 125.95 MiB | 32.69 MiB/s, done.
Resolving deltas: 100% (10664/10664), done.
/content/TTS
Note: checking out '4132240'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at 4132240 Merge branch 'dev' of https://github.com/mozilla/TTS into dev
     |████████████████████████████████| 320.4MB 45kB/s 
     |████████████████████████████████| 3.6MB 58.5MB/s 

In [ ]:
!cp -r "$DATAROOT/jsut_ljspeech_structure_22050.zip" /content/
!unzip -q /content/jsut_ljspeech_structure_22050.zip -d /content/

!cp -r "$DATAROOT/jsut_ver1.1_ljspeech_structure/phoneme_cache_4132240.zip" /content/
!unzip -q /content/phoneme_cache_4132240.zip -d /content/

!cp "$DATAROOT/jsut_ver1.1_ljspeech_structure/scale_stats.npy" /content/

In [ ]:
# load the default config file and update with the local paths and settings.
import json
from TTS.utils.io import load_config

DATAROOT = '/content/ljspeech_structure_22050'
DATAROOT_DRIVE ='/mydrive/machine-learning/tts/data/jsut_ver1.1_ljspeech_structure'

# glow tts用のconfigを元にする
CONFIG = load_config('/content/TTS/TTS/tts/configs/glow_tts_ljspeech.json') 

# 使うデータセットの設定
CONFIG['datasets'][0]['path'] = f"{DATAROOT}"
CONFIG['datasets'][0]['meta_file_train'] = 'metadata_no_han/metadata_kana_train.csv'
CONFIG['datasets'][0]['meta_file_val'] = 'metadata_no_han/metadata_kana_val.csv'
print(CONFIG['datasets'][0]['path'])
CONFIG['phoneme_language'] = 'ja'
CONFIG['phoneme_cache_path'] = "/content/phoneme_cache"
CONFIG['audio']['stats_path'] = "/content/scale_stats.npy"  # do not use mean and variance stats to normalizat spectrograms. Mean and variance stats need to be computed separately. 
CONFIG['audio']['signal_norm'] = True
CONFIG['text_cleaner'] = 'basic_cleaners'

# 出力先とテスト用データ
CONFIG['output_path'] = f"{DATAROOT_DRIVE}/output"
CONFIG['test_sentences_file'] = f"{DATAROOT_DRIVE}/test_sentences_file"

# 学習環境に応じて都合のよい値を入れる
CONFIG['num_loader_workers'] = 2
CONFIG['num_val_loader_workers'] = 1
CONFIG['print_step'] = 1000
CONFIG['save_step'] = 5000
#CONFIG['test_delay_epochs'] = 250

# create phoneme_cache
#CONFIG['phoneme_cache_path'] = f"{DATAROOT}/phoneme_cache"

with open('config.json', 'w') as fp:
    json.dump(CONFIG, fp)


/content/ljspeech_structure_22050


In [ ]:
# scale_stats.npyを作成するときだけ実行
%%script false --no-raise-error
!python ./TTS/bin/compute_statistics.py --config_path config.json --out_path  $DATAROOT_DRIVE/scale_stats.npy

In [ ]:
# 最初から学習を開始する場合
%%script false --no-raise-error
!CUDA_VISIBLE_DEVICES="0" python TTS/bin/train_glow_tts.py --config_path config.json

In [ ]:
# checkpointから学習を再開する場合
# outputのディレクトリ名を合わせる必要があるので注意
CONTINUE_PATH = f"{DATAROOT_DRIVE}/output/glow-tts-residual_bn_conv-jsut-stats"
!cp config.json $CONTINUE_PATH
!CUDA_VISIBLE_DEVICES="0" python TTS/bin/train_glow_tts.py --continue_path $CONTINUE_PATH

ストリーミング出力は最後の 5000 行に切り捨てられました。

 > TRAINING (2021-05-07 22:15:26) 

   --> TRAIN PERFORMACE -- EPOCH TIME: 165.99 sec -- GLOBAL_STEP: 393882
     | > avg_loss: -0.90841
     | > avg_log_mle: -0.95351
     | > avg_loss_dur: 0.04510
     | > avg_align_error: 0.00477
     | > avg_loader_time: 0.01259
     | > avg_step_time: 0.70904

 > EVALUATION 

 | > Synthesizing test sentences
  --> EVAL PERFORMANCE
     | > avg_loss: -0.81368 (+0.00157)
     | > avg_log_mle: -0.90570 (+0.00025)
     | > avg_loss_dur: 0.09201 (+0.00132)
     | > avg_align_error: 0.08297 (+0.00000)


 > EPOCH: 273/10000

 > TRAINING (2021-05-07 22:18:29) 

   --> STEP: 117/233 -- GLOBAL_STEP: 394000
     | > loss: -0.89408  (-0.92327)
     | > log_mle: -0.93872  (-0.95945)
     | > loss_dur: 0.04464  (0.03618)
     | > align_error: 0.00000  (0.00383)
     | > avg_spec_length: 353.7
     | > avg_text_length: 50.0
     | > step_time: 0.6476
     | > loader_time: 0.01
     | > current_lr: 0.00010075854437197567

   --> T

In [ ]:
# ljspeechの学習済みモデルを転移学習する場合
%%script false --no-raise-error
!CUDA_VISIBLE_DEVICES="0" python TTS/bin/train_glow_tts.py --config_path config.json --restore_path "$DATAROOT_DRIVE/../glow_tts_ljspeech_pretrained_best_model.pth.tar"

In [ ]:
%load_ext tensorboard
%tensorboard --logdir "$DATAROOT_DRIVE/output/glow-tts-residual_bn_conv-April-25-2021"